In [377]:
import numpy as np
import pandas as pd
import os
from checkRF_Score import *
from codes.utils.file import *

In [378]:
dataset_path = join_dir("dataset", "asteroid_dataset", "withils")

In [379]:
# in asteroid dataset
gt_file_name="true.true.geneTree.newick"
# gt_file_name = "raxml-ng.GTR+G.geneTree.newick"
species_tree_name="speciesTree.newick"
# species_tree_name= "fastrfs-raxml-ng_greedy.GTR+G.speciesTree.newick"

In [380]:
# for our experiment
gt_output_folder= "output"
gt_output_file_name= "1_gt.tre"
true_species_tree_name= "true-species.out.tree"

In [381]:
remove_dir_current(gt_output_folder)
create_dir_current(gt_output_folder)

In [382]:
import re

def remove_one(input_str):
    output_str = re.sub(r"1(?=[^\d_])", "", input_str)
    return output_str

def remove_numbers_after_colon(input_str):
    output_str = re.sub(r':[^,\)]+', '', input_str)
    output_str = remove_one(output_str)
    return output_str

def remove_numbers_after_colon_NO_NEED_TO_REMOVE_ONE(input_str):
    output_str = re.sub(r':[^,\)]+', '', input_str)
    return output_str

In [383]:
# provide families folder path
def get_all_GTs(families_folder):
    all_gts = []
    for dir in os.listdir(families_folder):
        # print(dir)
        # save mapping from mapping folder
        mapping={}
        mapping_dir="mappings"
        mapping_file_path=families_folder+"/"+dir+"/"+mapping_dir+"/treerecs_mapping.link"
        if os.path.exists(mapping_file_path):
            # print(mapping_file_path)
            with open(mapping_file_path) as f:
                for line in f:
                    # print(line)
                    (key, val) = line.split()
                    mapping[key] = val
        else:
            print("mapping file not found")
        if len(mapping) == 0:
            print("mapping is empty")
        # print(mapping)
        gt_tree_dir="gene_trees"
        gt_tree_file_path=families_folder+"/"+dir+"/"+gt_tree_dir+"/"+gt_file_name
        if not os.path.exists(gt_tree_file_path):
            print("file not exists")
            exit(1)
        with open(gt_tree_file_path) as f:
            line = f.readline()
            line = remove_numbers_after_colon(line)
            # print(line)
            for key in mapping:
                line = line.replace(key, mapping[key])
            all_gts.append(line)
            # print(line)
    #output to a file in output folder
    with open(gt_output_folder+"/"+gt_output_file_name, "w") as f:
        for gt in all_gts:
            f.write(gt+"\n")

In [384]:
def get_True_Species_Tree(source_folder):
    # copy species tree to output folder
    # species_tree_path = source_folder+"/species_trees/"+species_tree_name
    species_tree_path = join_dir(source_folder, "species_trees", species_tree_name)
    with open(species_tree_path) as f:
        line = f.readline()
        line = remove_numbers_after_colon_NO_NEED_TO_REMOVE_ONE(line)
        with open(gt_output_folder+"/"+true_species_tree_name, "w") as f:
            f.write(line)


In [385]:
def getTaxaSet(t):
    return t.replace('(', '').replace(')', '').split(';')[0].split(':')[0].split(',')

In [386]:
def get_INPUT_DATA():
    OUTGROUP = 'GAL'
    st = open('output/input.nwk')
    st = st.read()
    gts = open('output/1_gt.tre')

    gts = gts.read().split(';\n')[:-1] 

    # print(len(gts))

    # print(st)
    taxaSet = getTaxaSet(st)
    # print(taxaSet)
    matrix = np.zeros((len(taxaSet), len(gts)), dtype=int)


    # print(len(gts))
    for gt_idx, gt in  enumerate(gts) :
        # print(gt)
        cur_taxaSet = getTaxaSet(gt)
        # print(cur_taxaSet)
        for i, tx in enumerate(taxaSet):
            if tx  in cur_taxaSet or tx == OUTGROUP:
                matrix[i][gt_idx] = 1
    with open('output/input.data', 'w', newline='') as f:
        print(matrix.shape[0], matrix.shape[1], file=f)
        for i in range(len(taxaSet)):
            # f.write(matrix[i].tostring(), taxaSet[i])
            print(*matrix[i],taxaSet[i], end='\n', file=f)

In [387]:
def getTerrace():
    # !rm -rf terrace_output
    remove_dir_current("terrace_output")
    # !mkdir -p terrace_output
    create_dir_current("terrace_output")
    
    !./bin/raxml-ng output/input.nwk output/input.data > terrace_output/terrace_output.txt

In [388]:
def getConsensusTree():
    # get consensus tree
    terrace_dir = join_dir(TERRACE_OUTPUT_FOLDER)
    consensus_dir = join_dir(terrace_dir, "consensus")
    !rm -rf "$consensus_dir"
    !mkdir "$consensus_dir"

    if not dir_exists_abs(terrace_dir):
        print("Directory does not exist: {}".format(terrace_dir))
        exit(1)
    for terrace_file in os.listdir(terrace_dir):
        file_path = join_dir(terrace_dir, terrace_file)
        if file_path.endswith(".txt"):
            # print("Processing file: {}".format(file_path))
            terrace_file_name = terrace_file.split(".")[0]
            !./bin/raxml-ng --consense MRE --tree "$file_path" --prefix "$consensus_dir"/"$terrace_file_name"

In [389]:
def get_ASTRAL_tree():
    # !rm -rf in/
    remove_dir_current("in")
    # !rm -rf out/
    remove_dir_current("out")
    # !mkdir in
    create_dir_current("in")
    # !cp output/1_gt.tre in/
    copy_file_current("output/1_gt.tre", "in/1_gt.tre")
    
    !python3 postprocessor.py
    # !cp out/complete/astral/1_gt_species.out.tre output/input.nwk
    copy_file_current("out/complete/astral/1_gt_species.out.tre", f"{gt_output_folder}/input.nwk")

In [390]:
def get_True_ST_as_speciesTree():
    copy_file_current(f"{gt_output_folder}/{true_species_tree_name}", f"{gt_output_folder}/input.nwk")

In [391]:
def get_wQFM_tree():
    # !rm -rf in/
    remove_dir_current("in")
    # !rm -rf out/
    remove_dir_current("out")
    # !mkdir in
    create_dir_current("in")
    # !cp output/1_gt.tre in/
    copy_file_current("output/1_gt.tre", "in/1_gt.tre")
    
    !python3 postprocessor.py
    # !cp out/complete/astral/1_gt_species.out.tre output/input.nwk
    copy_file_current("out/complete/wQFM/1_gt_species.tre", "output/input.nwk")

In [392]:
def get_ASTRAL_RF_Score(true_tree):
    astral_rf_score = calculate_RF_distance_for_a_ST(join_dir(gt_output_folder, "input.nwk"), true_tree)
    print("astral_rf_score --> ", astral_rf_score)
    return astral_rf_score

In [393]:
def get_Terrace_RF_Scores(true_tree):
    
    #get RF score for each tree in Terrace
    terrace_rf_scores=calculate_RF_distance_for_terrace(join_dir(TERRACE_OUTPUT_FOLDER, "terrace_output.txt"), true_tree)
    # print("terrace_rf_scores", terrace_rf_scores)
    if terrace_rf_scores is not None:
        return terrace_rf_scores
    else:
        return []

In [394]:
def get_Consensus_RF_score(true_tree):
    terrace_dir = join_dir(TERRACE_OUTPUT_FOLDER)
    consensus_dir = join_dir(terrace_dir, "consensus")
    CONSENSUS_TREE_EXTENSION = "consensusTreeMRE"
    file_path = join_dir(consensus_dir, "terrace_output.raxml.consensusTreeMRE")
    # print("Processing file: {}".format(file_path))
    # print("True Tree --> ", true_tree)
    # remove_numbers_from_consensus_tree(file_path)
    if file_exists_absolute(file_path):
        dist = calculate_RF_distance_for_a_ST(file_path, true_tree)
        print("Consensus RF distance: {}".format(dist))
        return dist
    else:
        print("File does not exist: {}".format(file_path))
        return -1

In [395]:
def Keep_few_STs_in_Terrace(KEEPING_STs_NUM):
    num_of_trees_in_terrace = 0
    for file in os.listdir("terrace_output"):
        if file.endswith(".txt"):
            file_path = "terrace_output/"+file
            
            if os.path.getsize(file_path) <= 0:
                return -1 # no trees in terrace
            
            df = pd.read_csv(file_path)
            num_of_trees_in_terrace = len(df)
            if len(df) > KEEPING_STs_NUM:
                df = df.iloc[:KEEPING_STs_NUM]
                df.to_csv(file_path, index=False)
    return num_of_trees_in_terrace

In [396]:
def get_stats_of_terrace(terrace_rf_scores,true_tree, astral_rf):
    return calculate_RF_AND_generate_stats_OF_TERRACE_Return_better_trees(terrace_rf_scores,true_tree, astral_rf)

In [397]:
import datetime

csv_out = []
csv_out.append(["Model Condition", "RF Score", "Consensus RF" , "Better Trees Consensus RF Score", "Trees Better than ASTRAL", "Trees Equal to ASTRAL", "Tress worse than ASTRAL"])
i=0
MODEL_CONDITION = "s25"
for model_condition in sorted(os.listdir(dataset_path)):
    # if i==3:
        # i+=1
        # continue
    # print(model_condition)
    if MODEL_CONDITION not in model_condition:
        continue
    
    print("Model Condition --> ", model_condition)

    source_folder = join_dir(dataset_path, model_condition)
    # print(source_folder)
    # source_folder="dataset/asteroid_dataset/withils/ssim_veryhighmiss_s25_f1000_sites100_GTR_bl1.0_d0.0_l0.0_t0.0_gc0.0_p0.0_pop50000000_ms0.6_mf0.6_seed3000"
    families_folder = join_dir(source_folder, "families")
    get_all_GTs(families_folder)
    get_True_Species_Tree(source_folder)
    
    true_tree = join_dir(gt_output_folder, true_species_tree_name)
    
    # get_ASTRAL_tree()
    get_True_ST_as_speciesTree()

    get_INPUT_DATA()

    getTerrace()

    MAX_STs_IN_TERRACE = 20000 
    number_of_trees_in_terrace = Keep_few_STs_in_Terrace(MAX_STs_IN_TERRACE)
    if number_of_trees_in_terrace == -1:
        print("\n\n=============No trees in terrace================\n\n")
        continue
    #get rf scores of each tree in terrace
    terrace_file_path = join_dir(TERRACE_OUTPUT_FOLDER, "terrace_output.txt")
    # terrace_rf_scores = calculate_RF_distance_for_terrace(terrace_file_path, true_tree)
    
    # rf scores
    astral_rf = get_ASTRAL_RF_Score(true_tree)
    if number_of_trees_in_terrace >1:
        #consensus of trees of terrace
        getConsensusTree()
        
        consensus_rf = get_Consensus_RF_score(true_tree)

        # get stats of how many trees are better/eq/worse than ASTRAL
        better, equal, worse, better_trees = get_stats_of_terrace(terrace_file_path,true_tree, astral_rf)

        print(f"Better: {better}, Equal: {equal}, Worse: {worse}")
        
        rf_score_better_trees_consensus = None

        if better>1:
            # print("Better trees --> ", better_trees)

            better_trees_file_path = f'{gt_output_folder}/better_trees.txt'
            with open(better_trees_file_path, 'w') as f:
                f.writelines(better_trees)
            f.close()

            !./bin/raxml-ng --consense MRE --tree "$better_trees_file_path" --prefix "$gt_output_folder"/"better_trees"
            better_trees_consensus_file_path = f'{gt_output_folder}/better_trees.raxml.consensusTreeMRE'

            rf_score_better_trees_consensus = calculate_RF_distance_for_a_ST(better_trees_consensus_file_path, true_tree)
            # print("RF score of better trees consensus --> ", rf_score_better_trees_consensus)
            
        # csv_out.append([model_condition, astral_rf, consensus_rf])
        csv_out.append([model_condition, astral_rf, consensus_rf, rf_score_better_trees_consensus, better, equal, worse]) 
    
    i+=1
    if i==2:
        break

x = datetime.datetime.now()

stat_fl = f'stats/Summary_{MODEL_CONDITION}_{x.date()}_{x.time()}.csv'
with open(stat_fl, 'w') as stat_file:
    csv.writer(stat_file).writerows(csv_out)

# stat_fl = 'stats/Summary_terrace.csv'
# with open(stat_fl, 'w') as stat_file:
#     csv.writer(stat_file).writerows(csv_out_terrace)


Model Condition -->  ssim_veryhighmiss_s25_f1000_sites100_GTR_bl1.0_d0.0_l0.0_t0.0_gc0.0_p0.0_pop50000000_ms0.6_mf0.6_seed3000

================== ASTRAL ===================== 

This is ASTRAL version 5.6.3
Gene trees are treated as unrooted
478 trees read from /Users/sayemhasan/Desktop/Thesis/Terraces/raxml-ng/in/1_gt.tre
All output trees will be *arbitrarily* rooted at 24

======== Running the main analysis
Number of taxa: 25 (25 species)
Taxa: [24, 20, 9, 18, 6, 12, 3, 15, 4, 14, 22, 11, 17, 8, 21, 1, 7, 5, 13, 10, 2, 19, 23, 25, 16]
Taxon occupancy: {22=168, 23=29, 24=133, 25=102, 10=37, 11=85, 12=47, 13=47, 14=242, 15=169, 16=45, 17=197, 18=42, 19=65, 1=95, 2=37, 3=153, 4=172, 5=75, 6=191, 7=136, 8=108, 9=165, 20=187, 21=87}
Number of gene trees: 478
478 trees have missing taxa
Calculating quartet distance matrix (for completion of X)
Species tree distances calculated ...
Will attempt to complete bipartitions from X before adding using a distance matrix.
Building set of clusters (